In [1]:
from pathlib import Path
import pandas as pd
from fastai2.vision import *
from fastai2.optimizer import *
from fastai2.callback import *
import torch
DATA = Path('/content/clouderizer/panda/data')

TRAIN = DATA / 'train'
MASK = DATA / 'masks'
FOLDS = 8
EPOCHS = 16
N_TILES = 16
RRC = 1.0
FLIP = 1.0
NORM = 1.0
ROTATE = 1.0
LR = (1e-4, 1e-3)
MODEL_PATH = Path("/content/clouderizer/panda/out")
BATCH_SIZE = 128
VAL_BATCH_SIZE = 128

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv(DATA / 'train.csv')

In [3]:
df.head()

,image_id,data_provider,isup_grade,gleason_score
0,0005f7aaab2800f6170c399693a96917,karolinska,0,0+0
1,000920ad0b612851f8e01bcc880d9b3d,karolinska,0,0+0
2,0018ae58b01bdadc8e347995b69f99aa,radboud,4,4+4
3,001c62abd11fa4b57bf7a6c603a11bb9,karolinska,4,4+4
4,001d865e65ef5d2579c190a0e0350d8f,karolinska,0,0+0


In [4]:
df.shape[0]

10616

In [5]:
36 * df.shape[0]

382176

In [6]:
files = set(str(f) for f in TRAIN.iterdir())

In [7]:
len(files)

168256

In [72]:
sorted_files = [f'{TRAIN}/{f}_{i}.png' for f in sorted(df.image_id) for i in range(N_TILES)]

In [73]:
files = [f for f in sorted_files if f in files]

In [74]:
len(files)

168256

In [75]:
files[:5]

['/content/clouderizer/panda/data/train/0005f7aaab2800f6170c399693a96917_0.png',
 '/content/clouderizer/panda/data/train/0005f7aaab2800f6170c399693a96917_1.png',
 '/content/clouderizer/panda/data/train/0005f7aaab2800f6170c399693a96917_2.png',
 '/content/clouderizer/panda/data/train/0005f7aaab2800f6170c399693a96917_3.png',
 '/content/clouderizer/panda/data/train/0005f7aaab2800f6170c399693a96917_4.png']

In [77]:
mean = torch.tensor([1.0-0.90949707, 1.0-0.8188697, 1.0-0.87795304])
std = torch.tensor([0.36357649, 0.49984502, 0.40477625])